In [4]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini
랭스미스로 추적 중입니다 : lsv2_pt_24


# Model

생성형 언어 모델들은 입력된 문장을 통해 확률적으로 뒤에 오는 단어를 차례대로 예측합니다. 

### model 파라미터들

__주요 파라미터__
- temperature : 무작위성 조절 (0~1) --> 1에 가까울수록 다양하고 창의적인 답변, 0에 가까울수록 일관되고 정형화된 답변
- max_tokens : 출력 최대 토큰 수
- top_p : 누적 확률이 p가 될 때까지의 후보만 샘플링, p가 크면 다양한 단어가 나올 수 있음, temp만 써도 일반적으로 충분 --> 타 모델에서는 top_k가 있는 경우가 있음
---
__추가적인 파라미터__
- presence_penalty : 새 주제를 끌어들이도록 장려
- frequency_penalty : 이미 쓴 단어를 또 쓰는 정도를 억제
- max_retries : API 실패(일시적 오류, 레이트리밋) 시 자동 재시도 횟수

In [5]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,        # ai 파운드리에서 배포한 모델 이름   
    api_version="2024-12-01-preview",   # 사용할 API 버전
    temperature=0.2,                    # 무작위성 조절 (0~1)
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x127bf1d90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x127bf2890> root_client=<openai.lib.azure.AzureOpenAI object at 0x1272ea2d0> root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x127bf1a10> temperature=0.2 model_kwargs={} openai_api_key=SecretStr('**********') disabled_params={'parallel_tool_calls': None} azure_endpoint='https://kt-openai.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview' deployment_name='gpt-4.1-mini' openai_api_version='2024-12-01-preview' openai_api_type='azure'


In [6]:
# llm = AzureChatOpenAI(
#     api_key=AZURE_OPENAI_API_KEY,
#     azure_endpoint=END_POINT,
#     azure_deployment=MODEL_NAME,
#     api_version="2024-12-01-preview",
#     temperature=0.7,
#     max_tokens=512,     
#     top_p=0.9,          
#     frequency_penalty=0.2,
#     presence_penalty=0.1,
# )

In [7]:
for t in (0, 0.9):
    llm = AzureChatOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        azure_endpoint=END_POINT,  
        azure_deployment=MODEL_NAME,          
        api_version="2024-12-01-preview",
        temperature=t,
    )

    print(f'\n\nTemperature가 {t}일 때=========================')
    for i in range(3):
        print(f"\n[{i+1}번째 생성]")
        answer = llm.stream("어린왕자에 대해서 짧게 몇 문장으로 설명해줘!")
        for ans in answer:
            print(f"{ans.content}", end='', flush=True)



Temperature가 0일 때=========================

[1번째 생성]
《어린 왕자》는 프랑스 작가 생텍쥐페리가 쓴 동화 같은 소설로, 어린 왕자가 여러 행성을 여행하며 만난 사람들과의 이야기를 통해 사랑, 우정, 삶의 의미를 탐구하는 작품입니다. 순수한 시선으로 어른들의 세상을 비판하고, 진정한 가치가 무엇인지 생각하게 만듭니다.
[2번째 생성]
《어린 왕자》는 프랑스 작가 생텍쥐페리가 쓴 동화 같은 소설로, 어린 왕자가 여러 행성을 여행하며 만난 사람들과의 이야기를 통해 사랑, 우정, 삶의 의미를 탐구하는 작품입니다. 순수한 시선으로 어른들의 세상을 비판하고, 진정한 가치가 무엇인지 생각하게 만듭니다.
[3번째 생성]
《어린 왕자》는 프랑스 작가 생텍쥐페리가 쓴 동화 같은 소설로, 어린 왕자가 여러 행성을 여행하며 만난 사람들과의 이야기를 통해 사랑, 우정, 삶의 의미를 탐구하는 작품입니다. 순수한 시선으로 어른들의 세상을 비판하고, 진정한 가치가 무엇인지 생각하게 만듭니다.

Temperature가 0.9일 때=========================

[1번째 생성]
《어린 왕자》는 프랑스 작가 생텍쥐페리가 쓴 동화 같은 소설로, 어린 왕자가 여러 행성을 여행하며 순수함과 사랑, 삶의 의미를 탐구하는 이야기를 담고 있어요. 어른들의 세계를 비판하고 진정한 소중함에 대해 생각하게 만드는 작품입니다.
[2번째 생성]
《어린 왕자》는 프랑스 작가 생텍쥐페리가 쓴 소설로, 어른들의 세상에 대한 비판과 순수한 어린 시절의 시선을 담고 있습니다. 작은 왕자가 여러 행성을 여행하며 만나는 사람들과의 이야기를 통해 삶의 진정한 의미와 사랑, 우정의 가치를 전합니다. 간결한 문체와 깊은 철학적 메시지로 전 세계에서 사랑받는 작품입니다.
[3번째 생성]
《어린왕자》는 프랑스 작가 앙투안 드 생텍쥐페리가 쓴 동화 같은 철학적 소설로, 어린 왕자가 여러 행성을 여행하며 만난 인물들과 나눈 이야기들을 통해 사랑, 우정, 삶의 의미를 탐구한다. 순

## 토큰 수 계산하기

토큰은 usage_metadata에 기록되어 있다. reasoning이라는 토큰이 있는데 모델이 내부적으로 생각을 깊게한 경우 추가 토큰이 부여된다.

In [9]:
# invoke에서 토큰 계산
output = llm.invoke("어린왕자에 대해서 짧게 설명해줘.")
print(output.content)
print(output.usage_metadata) 

《어린왕자》는 프랑스 작가 생택쥐페리가 쓴 소설로, 순수함과 사랑, 우정에 대한 이야기를 담고 있습니다. 어린왕자가 여러 행성을 여행하며 만난 사람들과의 경험을 통해 삶의 진정한 의미를 되새기게 하는 작품입니다. 전 세계적으로 사랑받는 고전 문학입니다.
{'input_tokens': 20, 'output_tokens': 85, 'total_tokens': 105, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


stream모드에서는 제일 마지막에 토큰 정보가 포함되어 있다.

In [10]:
# stream에서 토큰 계산
answer = llm.stream("어린왕자에 대해서 짧게 몇 문장으로 설명해줘!", stream_usage=True)
for i, ans in enumerate(answer):
    print(f"{i}: {ans}")
    last_token = ans

print(f"\n\nTotal tokens: {last_token.usage_metadata}")

0: content='' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
1: content='' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
2: content='《' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
3: content='어' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
4: content='린' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
5: content=' 왕' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
6: content='자' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
7: content='》' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
8: content='는' additional_kwargs={} response_metadata={} id='run--6493f503-de21-4364-86f7-1b5d62bcc4a2'
9: content=' 프' additional_kwargs={} response_metadata={} id='run

### 다양한 LLM 모델들

1. Anthropic Claude (Sonnet 4 / Opus 4.1)
    - 추론·코딩 품질 상위권, 일반 모델이 200k의 초장문 컨텍스트, Sonnet 4는 500k. 

1. Meta Llama
    - 모델의 가중치를 공개함. 커스터마이징이 용이, 멀티 언어 지원이 좋은 편

1. Cohere Command R / R+
    - RAG같은 검색 기반 자료가 많음, 한국어 포함 다국어 지원 

1. DeepSeek (V3.1 / R1 Reasoner)
    - 매우 공격적인 가격과 추론 성능(R1), 일부 가중치/모델 공개. 

1. Alibaba Qwen (Qwen 2.5/3, QwQ-32B Reasoning)
    - 다국어·수학/코딩 특화 파생 모델 다수, 비주얼(Qwen-VL)도 활발, 한국어 성능도 괜찮은 편.

1. Amazon Titan (Bedrock)
    - AWS 생태계와 밀접한 통합성, 보안, 거버넌스, 인프라 운영 장점

## 로컬에서 모델 실행 - Ollama

In [11]:
!curl -fsSL https://ollama.com/install.sh | sh

ERROR: This script is intended to run on Linux only.


In [10]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 4.9 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   1% ▕                  ▏  13 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   1% ▕                  ▏  16 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   2% ▕                  ▏  24 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   2% ▕                  ▏  32 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   3% ▕                  ▏  36 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   3% ▕                  ▏  44 MB/1.3

In [ ]:
# 백그라운드로 띄우기
# !nohup ollama serve > /content/ollama.log 2>&1 &

# 기동 확인 (버전/태그 API가 응답하면 OK)
# !bash -lc 'for i in {1..20}; do curl -s http://localhost:11434/api/tags && break; sleep 1; done'

In [12]:
%pip install -q langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0.3,
)

print(llm.invoke("LangChain을 한 문장으로 설명해줘.").content)

LangChain은 언어의 구조를 다루는 도구로, 다양한 언어와 언어 간의 상호작용을 이해하는 데 도움이 됩니다.


In [ ]:
!ollama pull hf.co/mykor/Midm-2.0-Mini-Instruct-gguf

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="hf.co/mykor/Midm-2.0-Mini-Instruct-gguf",
    temperature=0.3,
)

print(llm.invoke("너는 어떤 모델이야? 너의 이름을 알아?").content)

나는 KT에서 개발한 AI 언어 모델인 믿:음(Mi:dm)입니다.


## Cache 적용

캐시(cache)란 '숨겨둔 저장소'라는 뜻으로 입력값과 출력을 저장해두었다가 똑같은 입력이 들어올 때 실제로 함수를 실행하지 않고 바로 출력을 하는 방법이다. 

아래 실행 예시를 보면 이후에는 total_cost가 0이다.

In [21]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 메모리 캐시
set_llm_cache(InMemoryCache())

In [26]:
import time
for i in range(1, 4):
    start = time.time()
    answer = llm.invoke("랭체인의 llm 캐시에 대해서 아주 짧게 설명해줘")
    print(f'[{i}번째]=========================')
    print(answer.content)
    print(f'>> {answer.usage_metadata}\n\n')
    print(f'>> {time.time() - start:.2f}초 소요')

[1번째]=========================
랭체인은 LLM(대규모 언어 모델)을 위한 프레임워크로, 여러 가지 기능을 제공합니다. 그 중 하나가 바로 'llm 캐시'입니다.

### llm 캐시란?

- **개념**: llm 캐시는 랭체인이 사용하는 대규모 언어 모델의 일부를 메모리에 저장해 두는 공간입니다.
  
- **목적**:
  - 빠른 응답 시간: 자주 사용되는 LLM의 일부를 캐시에 저장하여 필요할 때 빠르게 불러올 수 있습니다.
  - 메모리 효율성: 전체 모델을 메모리에 로드하지 않고 필요한 부분만 로드함으로써 시스템 자원을 절약할 수 있습니다.

### llm 캐시의 작동 방식

1. **모델 로딩**: 랭체인은 사용자가 특정 LLM을 사용할 때, 그 모델의 일부를 메모리에 로드합니다.
2. **캐시 저장**: 이 로드된 데이터는 캐시에 저장됩니다.
3. **요청 처리**: 이후 동일한 LLM을 요청할 경우, 캐시는 저장된 데이터를 빠르게 반환하여 응답 시간을 단축시킵니다.

### llm 캐시의 장점

- **성능 향상**: 자주 사용하는 LLM의 일부만 로드하므로, 전체 모델을 다시 로드하는 것보다 훨씬 빠릅니다.
- **자원 절약**: 메모리 사용량이 줄어들어 시스템 자원을 효율적으로 사용할 수 있습니다.
- **응답 속도 개선**: 사용자에게 더 빠른 응답 시간을 제공합니다.

### llm 캐시의 한계

- **제한된 범위**: 캐시는 LLM의 일부만 저장할 수 있으며, 모델 전체를 저장하는 것은 아닙니다. 따라서 복잡한 작업이나 대량의 데이터 처리에는 적합하지 않을 수 있습니다.
- **업데이트 문제**: LLM이 업데이트될 경우, 캐시에 저장된 부분도 함께 업데이트되어야 합니다.

랭체인의 llm 캐시는 이러한 방식으로 시스템 성능을 개선하고 자원 사용 효율성을 높이는 데 기여합니다.
>> {'input_tokens': 23, 'output_tokens': 361, 'total_tokens': 384}


>> 3.71